# Veeam API and Python Part 6 - Veeam API and Pandas
In this notebook we will go over how you can use Python's Pandas package to manipulate API responses

You will need to install the following:

    pip install pandas random-word

First the imports:

In [273]:
from veeam_easy_connect import VeeamEasyConnect
import requests
import urllib3
urllib3.disable_warnings()
# new
import pandas as pd

### What is Pandas?

Pandas is a data analysis framework that makes manipulating large datasets very easy.

[Pandas](https://pandas.pydata.org/)

It uses Dataframes which sounds complicated but they are essentially virtual Excel style tables.

|| col1 | col2 | col3 | col4 |
|-|------|------|------|------|
| row 1| data1 | data2 | data3 | data4 |
| row 2 | data5 | data6 | data7 | data8 |

You can use this structure with SQL like queries to filter, modify, and transform.

We can create Dataframes from scratch doing following.

In [274]:
# create some random words to use in the dataframe
from random_word import RandomWords

r = RandomWords()

ran_words = r.get_random_words(hasDictionaryDef="true")
r_words = ran_words[:10]

In [275]:
import random

# creating some random data
list1 = random.sample(range(1, 25), 10)
list2 = random.sample(range(3, 50), 10)

# create a dataframe
dataframe = pd.DataFrame({
    "col1" : list1,
    "col2" : list2},
    index=r_words
    )

In [276]:
# print the dataframe
dataframe

,col1,col2
racket-ground,13,9
destined,14,42
thatched,20,36
intermetallic,12,31
butcha,10,29
satins,11,46
bread-barge,23,40
pauline,22,44
unwearable,19,34
genericness,18,22


There are a LOT of ways you can work with the data, but for context here are a few examples.

First filtering using iloc which syntax is as follows .iloc[start_row:end_row, start_col:end_col]

Below you will see we are going for the 3rd > 5th row and all the columns

In [277]:
dataframe.iloc[3:5, :] 

,col1,col2
intermetallic,12,31
butcha,10,29


Next we can do math on the data, assign it to a new dataframe then print

In [278]:
df2= dataframe * 100
df2

,col1,col2
racket-ground,1300,900
destined,1400,4200
thatched,2000,3600
intermetallic,1200,3100
butcha,1000,2900
satins,1100,4600
bread-barge,2300,4000
pauline,2200,4400
unwearable,1900,3400
genericness,1800,2200


In [303]:
# show values over 15
dataframe[dataframe['col1'] > 15]

,col1,col2
thatched,20,36
bread-barge,23,40
pauline,22,44
unwearable,19,34
genericness,18,22


Create a new column that provides a cumulative sum value

In [280]:
df2['col3'] = df2['col2'].cumsum()
df2

,col1,col2,col3
racket-ground,1300,900,900
destined,1400,4200,5100
thatched,2000,3600,8700
intermetallic,1200,3100,11800
butcha,1000,2900,14700
satins,1100,4600,19300
bread-barge,2300,4000,23300
pauline,2200,4400,27700
unwearable,1900,3400,31100
genericness,1800,2200,33300


In [304]:
df2.to_excel("df_data.xlsx")

## Next to apply this to the Veeam API

Next log into the Enterprise Manager API using Veeam Easy Connect

In [281]:
veeam_ec = VeeamEasyConnect()
header = veeam_ec.em_login()

Create the get function

In [282]:
def get_data(url: str, headers: dict) -> dict:
    res = requests.get(url, headers=headers, verify=False)
    res.raise_for_status() # will raise an error if we don't get a 2** status code
    res_data = res.json()
    return res_data

Grab an endpoint from the Veeam Documentation, in this case we will use the **/query?type=BackupFile** endpoint.

[/querySvc backupFile - Help Guide](https://helpcenter.veeam.com/docs/backup/em_rest/get_query_backupfile.html?ver=110)

Note that the &format=Entities means we will get back all the backup file information in a single call.

In [283]:
address = input("Enter Server Address: ")
url = f"https://{address}:9398/api/query?type=BackupFile&format=Entities"

Then make a request to get the data

In [284]:
bu_data = get_data(url, header)

## Start applying this to the Veeam API

First we will filter out the data that we want from the backup file response

In [285]:
bu_data.keys()

dict_keys(['Refs', 'Entities', 'Resources', 'Links', 'PagingInfo'])

In [286]:
bu_data['Entities'].keys()

dict_keys(['Jobs', 'FailoverPlans', 'Backups', 'Replicas', 'Repositories', 'RestorePoints', 'VmRestorePoints', 'VAppRestorePoints', 'VmReplicaPoints', 'BackupJobSessions', 'ReplicaJobSessions', 'ReplicaTaskSessions', 'RestoreSessions', 'HierarchyRoots', 'BackupTaskSessions', 'BackupServers', 'ManagedServers', 'EnterpiseRoles', 'EnterpiseAccounts', 'WanAccelerators', 'CloudGateways', 'CloudTenants', 'CloudReplicas', 'VCloudOrganizationConfigs', 'BackupFiles', 'VSphereSelfServiceConfigs', 'CloudPublicIps', 'CloudHardwarePlans', 'CloudVmReplicaPoints', 'AgentRestorePoints', 'AgentBackupJob', 'CloudGatewayPools', 'CloudFailoverPlans', 'ExternalRepositories', 'VlanConfigurations', 'CloudFailoverSessions', 'AgentProtectionGroups', 'DiscoveredComputers', 'FileServers', 'NASJobs', 'CdpReplicas', 'CdpReplicaVms', 'CdpPolicies', 'CdpReplicaSessions', 'CdpReplicaTaskSessions', 'VAppReplicaPoints'])

We will grab the key capacity data from the endpoint

In [287]:
bu_data['Entities']['BackupFiles']['BackupFiles'][0].keys()

dict_keys(['FilePath', 'BackupSize', 'DataSize', 'DeduplicationRatio', 'CompressRatio', 'CreationTimeUtc', 'FileType', 'Name', 'UID', 'Links', 'Href', 'Type'])

In [288]:
filter_bu = []
# creating a new variable for just the backup files
bu_data2 = bu_data['Entities']['BackupFiles']['BackupFiles']

for i in bu_data2:
    data = {
        "BackupSize": i["BackupSize"],
        "DataSize": i["DataSize"],
        "DeduplicationRatio": i["DeduplicationRatio"],
        "CompressRatio": i["CompressRatio"],
        "CreationTimeUtc": i["CreationTimeUtc"],
        "FileType": i["FileType"],
        "jobName": i["Links"][0]["Name"]
    }
    filter_bu.append(data)

The data we have now is 'flat' in that we have don't have nested dictionaries or lists.

Below we can see a single entry in the list we created

In [289]:
filter_bu[0]

{'BackupSize': 40292352,
 'DataSize': 219798776,
 'DeduplicationRatio': 1.0,
 'CompressRatio': 5.56,
 'CreationTimeUtc': '2021-09-03T18:00:18Z',
 'FileType': 'vib',
 'jobName': 'solaris solaris_job'}

We are going now create a Dataframe from this data. You will see that this is very easy to do.

In [290]:
bu_df = pd.DataFrame.from_dict(filter_bu)

In [291]:
bu_df.head()

,BackupSize,DataSize,DeduplicationRatio,CompressRatio,CreationTimeUtc,FileType,jobName
0,40292352,219798776,1.00,5.56,2021-09-03T18:00:18Z,vib,solaris solaris_job
1,9074769920,43589968582,3.17,1.52,2021-06-26T17:00:26.123Z,vbk,Agent to AWS - dc1backupvm3.testlab.net
2,20492288,22020096,1.00,1.59,2021-04-15T15:37:46Z,vib,Mac Backup - Edwards-MacBook-Air.local
3,778895360,8589979960,4.96,2.22,2021-08-29T21:00:35.817Z,vbk,PerJob
4,370257920,838892876,1.00,2.27,2021-06-27T16:00:37.733Z,vib,Agent Backup Job 1 - dc1backupvm3.testlab.net


The BackupSize and DataSize values are in bytes so we need to change these to something more manageable.

In [292]:
bu_df['BackupSizeGB'] = bu_df['BackupSize'] / 1024**3
bu_df['DataSizeGB'] = bu_df['DataSize'] / 1024**3

In [293]:
bu_df.head()
# .head() shows just the first few items, you can also use .tail() 

,BackupSize,DataSize,DeduplicationRatio,CompressRatio,CreationTimeUtc,FileType,jobName,BackupSizeGB,DataSizeGB
0,40292352,219798776,1.00,5.56,2021-09-03T18:00:18Z,vib,solaris solaris_job,0.037525,0.204704
1,9074769920,43589968582,3.17,1.52,2021-06-26T17:00:26.123Z,vbk,Agent to AWS - dc1backupvm3.testlab.net,8.451538,40.596322
2,20492288,22020096,1.00,1.59,2021-04-15T15:37:46Z,vib,Mac Backup - Edwards-MacBook-Air.local,0.019085,0.020508
3,778895360,8589979960,4.96,2.22,2021-08-29T21:00:35.817Z,vbk,PerJob,0.725403,8.000042
4,370257920,838892876,1.00,2.27,2021-06-27T16:00:37.733Z,vib,Agent Backup Job 1 - dc1backupvm3.testlab.net,0.344830,0.781280


You can drop columns with the .drop method [link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html?highlight=drop#pandas.DataFrame.drop)

We can now do a quick bit of analysis, let's filter out the vib and calculate the average Backup Size

In [294]:
bu_vib = bu_df[bu_df['FileType'] == "vib"].copy()

In [295]:
bu_vib['DataSizeGB'].mean()

1.723814519910285

In [305]:
bu_vib['DataSizeGB'].max()

13.80764176696539

Next we can have a look at the average compression per Job

Check out the docs on [.loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)

In [297]:
# first change the ratio to a percentage
bu_vib.loc[:, "CompressPercentage"] = bu_vib["CompressRatio"] * 100

In [298]:
bu_vib.groupby("jobName")[['CompressRatio', 'CompressPercentage']].mean().round(2).sort_values("CompressRatio", ascending=False)

,CompressRatio,CompressPercentage
jobName,,
Mixed SOBR,7.14,714.00
k8s,5.75,574.70
solaris solaris_job,5.56,556.00
PerJob,5.44,543.71
SQL1,3.01,300.57
Agent Backup Job 1 - dc1backupvm3.testlab.net,2.38,237.86
Agent to AWS - dc1backupvm3.testlab.net,2.21,221.43
Tiny NFS vm,2.17,217.00
Mac Backup - Edwards-MacBook-Air.local,1.67,167.17
